# Theme

This project simulates an industrial factory robot that can move around a map either manually or through AI-generated commands using Gemini, visually showing its position, path, and actions in real time.

# Install

gradio → Builds the interactive dashboard for controlling and visualizing the robot.

langchain-google-genai → Connects LangChain with Gemini so AI can interpret commands and plan robot movements.

langchain → Handles structured prompts, messages, and reasoning pipelines for the AI agent.

In [ ]:
!pip install -q gradio langchain-google-genai langchain

Gemini API Key set up

Step-1: Go to Google AI Studio https://ai.google.dev/?utm_source=chatgpt.com

Step-2: Sign in with your Google account.

Step-3: Open the “API Keys” page. In the left sidebar, click on “API keys”.

Step-4: Create or select a project.

Step-5: Create a new API key.

Step-6: Copy and store the key safely.

RobotState Class Cell → Stores the robot's coordinates, gripper status, and movement history across UI interactions.

draw_factory_map() Cell → Draws the robot, landmarks, and movement path on a fixed factory map using Matplotlib.

manual_move() Cell → Updates the robot’s position instantly when a user adjusts the sliders.

ai_move() Cell → Sends user commands to Gemini, receives JSON instructions, and moves the robot accordingly.

Gradio UI Cell → Builds the two control modes—Manual and AI—and displays the live factory map and system logs.

demo.launch() Cell → Starts the complete robot control center allowing real-time interaction.

In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import json
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage

# --- 1. SETUP API ---
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('Insert your API Key') #Insert Your API KEY
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0)
except:
    print("⚠️ Key not found. AI features won't work without 'gem_Key' in Secrets.")
    llm = None

# --- 2. ROBOT STATE ---
# We keep the robot state global so it persists between Gradio updates
class RobotState:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.gripper = "OPEN"
        self.history = [] # Stores (x, y) tuples for the path line

    def update(self, x, y):
        self.x = x
        self.y = y
        self.history.append((x, y))
        # Keep history manageable (last 50 moves)
        if len(self.history) > 50: self.history.pop(0)

robot = RobotState()

# --- 3. PLOTTING FUNCTION ---
def draw_factory_map(current_x, current_y, show_path=True):
    # Create a consistent figure size
    fig, ax = plt.subplots(figsize=(6, 4))

    # Fixed Limits so the "room" doesn't resize
    ax.set_xlim(-2, 25)
    ax.set_ylim(-2, 25)

    # Draw Landmarks
    landmarks = {"Home": (0, 0), "Conveyor": (10, 5), "Pallet": (5, 20)}
    for name, (lx, ly) in landmarks.items():
        ax.scatter(lx, ly, s=200, c='lightgray', edgecolors='black')
        ax.text(lx, ly-1.5, name, ha='center', fontsize=9)

    # Draw Path History
    if show_path and len(robot.history) > 1:
        hist_x = [p[0] for p in robot.history]
        hist_y = [p[1] for p in robot.history]
        ax.plot(hist_x, hist_y, '--', color='gray', alpha=0.5)

    # Draw Robot (Current Position)
    # Color changes based on Gripper State (Green=Open, Red=Closed)
    color = 'green' if robot.gripper == "OPEN" else 'red'
    ax.scatter(current_x, current_y, s=300, c=color, edgecolors='black', zorder=10)
    ax.text(current_x, current_y+1, "ROBOT", ha='center', fontweight='bold', color='blue')

    ax.grid(True, linestyle=':', alpha=0.6)
    ax.set_title(f"Position: ({current_x}, {current_y}) | Gripper: {robot.gripper}")
    return fig

# --- 4. GRADIO LOGIC ---

# A. Manual Slider Logic
def manual_move(x, y):
    robot.update(x, y)
    return draw_factory_map(x, y)

# B. AI Logic
def ai_move(prompt):
    if not llm: return None, "⚠️ API Key Missing"

    system_prompt = """
    You are a Robot Controller.
    Map: Home(0,0), Conveyor(10,5), Pallet(5,20).
    Output JSON ONLY: [{"action": "move_to", "params": {"x": 10, "y": 5}}]
    """

    try:
        # Ask Gemini
        messages = [SystemMessage(content=system_prompt), HumanMessage(content=prompt)]
        res = llm.invoke(messages)

        # Parse JSON
        clean = res.content.replace("```json", "").replace("```", "").strip()
        plan = json.loads(clean)

        status_log = f"🧠 Plan: {plan}\n"

        # Execute only the LAST move to show final state (Animation is hard in static plots)
        # In a real app, you'd yield frames here.
        for step in plan:
            if step["action"] == "move_to":
                robot.update(step["params"]["x"], step["params"]["y"])
            elif step["action"] == "set_gripper":
                val = step["params"]
                robot.gripper = val.get("state") if isinstance(val, dict) else val

        return draw_factory_map(robot.x, robot.y), status_log

    except Exception as e:
        return draw_factory_map(robot.x, robot.y), f"❌ Error: {str(e)}"

# --- 5. BUILD UI ---
with gr.Blocks() as demo:
    gr.Markdown("## 🏭 Industrial Robot Control Center")

    with gr.Row():
        # Left Column: The Map
        plot_output = gr.Plot(label="Live Factory Map")

    with gr.Tabs():
        # TAB 1: MANUAL CONTROL
        with gr.TabItem("🎮 Manual Control"):
            gr.Markdown("Use sliders to move the robot in real-time.")
            with gr.Row():
                slider_x = gr.Slider(0, 25, value=0, step=1, label="X Coordinate")
                slider_y = gr.Slider(0, 25, value=0, step=1, label="Y Coordinate")

            # Connect sliders to function
            # 'live=True' updates the graph while dragging
            slider_x.change(manual_move, inputs=[slider_x, slider_y], outputs=plot_output)
            slider_y.change(manual_move, inputs=[slider_x, slider_y], outputs=plot_output)

        # TAB 2: AI CONTROL
        with gr.TabItem("🤖 AI Commander"):
            gr.Markdown("Type a command like: 'Go to Conveyor and pick up item'")
            ai_input = gr.Textbox(label="Command")
            ai_btn = gr.Button("Execute", variant="primary")
            ai_status = gr.Textbox(label="System Logs", lines=2)

            ai_btn.click(ai_move, inputs=[ai_input], outputs=[plot_output, ai_status])

# Launch
demo.launch(debug=False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 4.29.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://c8a7875e07f38e8cfe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


✔️ Live robot map
✔️ Robot path drawn
✔️ AI executes movement
✔️ Status logs explaining action